In [1]:
import pandas as pd
import numpy as np
import sys
from scipy.spatial.distance import cityblock
from progressbar import ProgressBar

In [2]:
def core_tcrf(core):
    core_tcrf = f'___{core[3]}{core[4]}{core[5]}{core[6]}{core[7]}_'
    return core_tcrf

In [13]:
## input is the netMPHpan xlsx for multiple sequence but single allel
## for single allel, load the xls file and filter by cutoff
#netMHCpan_out="netMHCpan_out copy/RSVA_F_test.xls"
netMHCpan_out="snakemake/results/RSVA_test.xls"
data = pd.read_csv(netMHCpan_out,sep="\t",skiprows=0)
## get core position 
df_epitope = data.loc[data['EL_Rank'] <2] 

In [14]:
df_epitope['core_tcrf'] = df_epitope.apply(lambda x:core_tcrf(x['core']),axis=1)

df_epitope

/var/folders/3n/dzwnf1yn2pz2qt6dgklrqmfh0000gp/T/ipykernel_73442/1000648111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epitope['core_tcrf'] = df_epitope.apply(lambda x:core_tcrf(x['core']),axis=1)


,Pos,Peptide,ID,core,icore,EL-score,EL_Rank,Ave,NB,core_tcrf
22,22,ASSQNITEEFY,KF826828-A-2004,ASSQTEEFY,ASSQNITEEFY,0.5147,0.2796,0.5147,1,___QTEEF_
33,33,QSTCSAVSKGY,KF826828-A-2004,QSSAVSKGY,QSTCSAVSKGY,0.0916,1.3607,0.0916,1,___AVSKG_
187,187,LTSKVLDLKNY,KF826828-A-2004,LTSKVLDLY,LTSKVLDLKNY,0.3065,0.5307,0.3065,1,___KVLDL_
239,239,NAGVTTPVSTY,KF826828-A-2004,NATTPVSTY,NAGVTTPVSTY,0.0786,1.5013,0.0786,1,___TPVST_
288,288,MSIIKEEVLAY,KF826828-A-2004,MSIEEVLAY,MSIIKEEVLAY,0.2965,0.5475,0.2965,1,___EEVLA_
...,...,...,...,...,...,...,...,...,...,...
22022,535,VILLALIAV,KY883567-A-2015,VILLALIAV,VILLALIAV,0.1171,1.6735,0.1171,1,___LALIA_
22024,537,LLALIAVGL,KY883567-A-2015,LLALIAVGL,LLALIAVGL,0.2465,0.9387,0.2465,1,___LIAVG_
22026,539,ALIAVGLLL,KY883567-A-2015,ALIAVGLLL,ALIAVGLLL,0.4151,0.5080,0.4151,1,___AVGLL_
22189,136,FLGFLLGV,KY883567-A-2015,FL-GFLLGV,FLGFLLGV,0.1356,1.5165,0.1356,1,___GFLLG_


In [15]:
df_epitope = df_epitope[["ID","Pos", "core_tcrf","EL-score"]]
df_epitope

,ID,Pos,core_tcrf,EL-score
22,KF826828-A-2004,22,___QTEEF_,0.5147
33,KF826828-A-2004,33,___AVSKG_,0.0916
187,KF826828-A-2004,187,___KVLDL_,0.3065
239,KF826828-A-2004,239,___TPVST_,0.0786
288,KF826828-A-2004,288,___EEVLA_,0.2965
...,...,...,...,...
22022,KY883567-A-2015,535,___LALIA_,0.1171
22024,KY883567-A-2015,537,___LIAVG_,0.2465
22026,KY883567-A-2015,539,___AVGLL_,0.4151
22189,KY883567-A-2015,136,___GFLLG_,0.1356


In [16]:
## groupby and calculate sum to see 
df_epitope=df_epitope.groupby(["ID","Pos", "core_tcrf"])["EL-score"].sum().reset_index()
df_epitope

,ID,Pos,core_tcrf,EL-score
0,KF530260-A-2005,6,___AITTI_,0.0670
1,KF530260-A-2005,9,___TIFAA_,0.2259
2,KF530260-A-2005,22,___QTEEF_,0.5147
3,KF530260-A-2005,23,___NITEE_,0.9186
4,KF530260-A-2005,24,___ITEEF_,0.2447
...,...,...,...,...
511,KY883567-A-2015,537,___LIAVG_,0.2465
512,KY883567-A-2015,538,___VGLLL_,0.0599
513,KY883567-A-2015,539,___AVGLL_,0.4151
514,KY883567-A-2015,539,___VGLLL_,0.1291


In [17]:
#https://www.statology.org/manhattan-distance-python/
# use mahattan distance to see what happens?    
## reshape dataframe
epitope_feature = df_epitope.pivot_table(index=['core_tcrf'], columns=['ID'],values="EL-score")
epitope_feature = epitope_feature.fillna(0)
## calculate mahatton distance

epitope_feature

ID,KF530260-A-2005,KF826828-A-2004,KF826838-A-2006,KF826841-A-2007,KY883567-A-2015
core_tcrf,,,,,
___-GNTL_,0.1002,0.1002,0.1002,0.1002,0.1002
___-LQLL_,0.0774,0.0774,0.0774,0.0774,0.0774
___-NTLY_,0.1225,0.1225,0.1225,0.1225,0.1225
___-SLIA_,0.0000,0.0000,0.1217,0.0000,0.0000
___-YVVQ_,0.2407,0.2407,0.2407,0.2407,0.2407
...,...,...,...,...,...
___VKGEP_,0.1577,0.1577,0.1577,0.1577,0.1577
___VLDLK_,0.1618,0.1618,0.1618,0.1618,0.1618
___VRQQS_,0.0825,0.0825,0.0825,0.0825,0.0825


In [18]:
epitope_feature.to_csv("epitope_feature_test.csv",index=False)

In [19]:
## get sequence ID
seq = df_epitope.ID.unique()
seq
pbar = ProgressBar()
## loop through all seq in seq array to calculate Manhatton distance
seq_distance=[]
for i in pbar(range(len(seq))):
    for j in range(len(seq)):
        if i<j:
            distance = cityblock(epitope_feature[seq[i]], epitope_feature[seq[j]])
            seq_distance.append([seq[i], seq[j], distance])                
            df_distance = pd.DataFrame(seq_distance, columns=["seq1", "seq2","epitope_distance"])           
df_distance

100% |##################################################################################################################|


,seq1,seq2,epitope_distance
0,KF530260-A-2005,KF826828-A-2004,0.917700
1,KF530260-A-2005,KF826838-A-2006,2.774267
2,KF530260-A-2005,KF826841-A-2007,1.023900
3,KF530260-A-2005,KY883567-A-2015,4.678033
4,KF826828-A-2004,KF826838-A-2006,2.728167
5,KF826828-A-2004,KF826841-A-2007,0.106200
6,KF826828-A-2004,KY883567-A-2015,4.631933
7,KF826838-A-2006,KF826841-A-2007,2.834367
8,KF826838-A-2006,KY883567-A-2015,3.262433
9,KF826841-A-2007,KY883567-A-2015,4.738133


In [20]:
def seq_mandist(df_epitope,epitope_feature):
    ## get sequence ID
    
    seq = df_epitope.ID.unique()
    ## loop through all seq in seq array to calculate Manhatton distance
    pbar = ProgressBar()
    seq_distance=[]
    for i in pbar(range(len(seq))):
        for j in range(len(seq)):
            if i<j:
                distance = cityblock(epitope_feature[seq[i]], epitope_feature[seq[j]])
                seq_distance.append([seq[i], seq[j], distance])                
                distance_df = pd.DataFrame(seq_distance, columns=["seq1", "seq2","epitope_distance"])
                    
    return(distance_df)

In [21]:
seq_epidist=seq_mandist(df_epitope,epitope_feature)

100% |##################################################################################################################|


In [22]:
seq_epidist

,seq1,seq2,epitope_distance
0,KF530260-A-2005,KF826828-A-2004,0.917700
1,KF530260-A-2005,KF826838-A-2006,2.774267
2,KF530260-A-2005,KF826841-A-2007,1.023900
3,KF530260-A-2005,KY883567-A-2015,4.678033
4,KF826828-A-2004,KF826838-A-2006,2.728167
5,KF826828-A-2004,KF826841-A-2007,0.106200
6,KF826828-A-2004,KY883567-A-2015,4.631933
7,KF826838-A-2006,KF826841-A-2007,2.834367
8,KF826838-A-2006,KY883567-A-2015,3.262433
9,KF826841-A-2007,KY883567-A-2015,4.738133


In [ ]:
def seq_mandist(df_epitope,epitope_feature):
    ## get sequence ID
    
    seq = df_epitope.ID.unique()
    ## loop through all seq in seq array to calculate Manhatton distance
    pbar = ProgressBar()
    seq_distance=[]
    for i in pbar(range(len(seq))):
        for j in range(len(seq)):
            if i<j:
                distance = cityblock(epitope_feature[seq[i]], epitope_feature[seq[j]])
                seq_distance.append([seq[i], seq[j], distance])                
                df_distance = pd.DataFrame(seq_distance, columns=["seq1", "seq2","epitope_distance"])
                    
    return(df_distance)
    